In [1]:
import sys
import os
import os.path as osp
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import ecole
import json
from torch_geometric.data import (InMemoryDataset, Data)
from torch_geometric.data import DataLoader

/home/leonaire/anaconda3/envs/ecole/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
import sys

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '.')

import torch
import torch.nn.functional as F
from torch.nn import BatchNorm1d as BN
from torch.nn import Sequential, Linear, ReLU
from torch_geometric.nn import MessagePassing

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class SimpleBipartiteLayer(MessagePassing):
    def __init__(self, edge_dim, dim, aggr):
        super(SimpleBipartiteLayer, self).__init__(aggr=aggr, flow="source_to_target")

        self.nn = Sequential(Linear(3 * dim, dim), ReLU(), Linear(dim, dim), ReLU(),
                             BN(dim))

        # Maps edge features to the same number of components as node features.
        self.edge_encoder = Sequential(Linear(edge_dim, dim), ReLU(), Linear(dim, dim), ReLU(),
                                       BN(dim))

    def forward(self, source, target, edge_index, edge_attr, size):
        # Map edge features to embeddings with the same number of components as node embeddings.
        edge_embedding = self.edge_encoder(edge_attr)

        out = self.propagate(edge_index, x=source, t=target, edge_attr=edge_embedding, size=size)

        return out

    def message(self, x_j, t_i, edge_attr):
        return self.nn(torch.cat([t_i, x_j, edge_attr], dim=-1))

    def __repr__(self):
        return '{}(nn={})'.format(self.__class__.__name__, self.nn)


class SimpleNet(torch.nn.Module):
    def __init__(self, hidden, aggr, num_layers, regression=False):
        super(SimpleNet, self).__init__()
        self.num_layers = num_layers

        self.regression = regression

        # Embed initial node features.
        self.var_node_encoder = Sequential(Linear(2, hidden), ReLU(), Linear(hidden, hidden))
        self.con_node_encoder = Sequential(Linear(2, hidden), ReLU(), Linear(hidden, hidden))

        # Bipartite GNN architecture.
        self.layers_con = []
        self.layers_var = []
        for i in range(self.num_layers):
            self.layers_con.append(SimpleBipartiteLayer(1, hidden, aggr=aggr))
            self.layers_var.append(SimpleBipartiteLayer(1, hidden, aggr=aggr))

        self.layers_con = torch.nn.ModuleList(self.layers_con)
        self.layers_var = torch.nn.ModuleList(self.layers_var)

        # MLP used for classification.
        self.lin1 = Linear((num_layers + 1) * hidden, hidden)
        self.lin2 = Linear(hidden, hidden)
        self.lin3 = Linear(hidden, hidden)
        if not self.regression:
            self.lin4 = Linear(hidden, 2)
        else:
            self.lin4 = Linear(hidden, 1)

    def forward(self, data):

        # Get data of batch.
        var_node_features = data.var_node_features
        con_node_features = data.con_node_features
        edge_index_var = data.edge_index_var
        edge_index_con = data.edge_index_con
        edge_features_var = data.edge_features_var
        edge_features_con = data.edge_features_con
        num_nodes_var = data.num_nodes_var
        num_nodes_con = data.num_nodes_con

        # Compute initial node embeddings.
        var_node_features_0 = self.var_node_encoder(var_node_features)
        con_node_features_0 = self.con_node_encoder(con_node_features)

        x_var = [var_node_features_0]
        x_con = [con_node_features_0]

        num_var = var_node_features_0.size(0)
        num_con = con_node_features_0.size(0)

        for i in range(self.num_layers):
            x_con.append(F.relu(self.layers_var[i](x_var[-1], x_con[-1], edge_index_var, edge_features_var,
                                                   (num_var, num_con))))

            x_var.append(F.relu(self.layers_con[i](x_con[-1], x_var[-1], edge_index_con, edge_features_con,
                                                   (num_con, num_var))))

        x = torch.cat(x_var[:], dim=-1)

        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = self.lin4(x)

        if not self.regression:
            return F.log_softmax(x, dim=-1)
        else:
            return x.view(-1)

    def __repr__(self):
        return self.__class__.__name__


In [98]:
bias_threshold = 0.5

# Preprocess indices of bipartite graphs to make batching work.
class MyData(Data):
    def __inc__(self, key, value,store):
        if key in ['edge_index_var']:
            return torch.tensor([self.num_nodes_var, self.num_nodes_con]).view(2, 1)
        elif key in ['edge_index_con']:
            return torch.tensor([self.num_nodes_con, self.num_nodes_var]).view(2, 1)
        elif key in ['index']:
            return torch.tensor(self.num_nodes_con)
        elif key in ['index_var']:
            return torch.tensor(self.num_nodes_var)
        else:
            return 0


class MyTransform(object):
    def __call__(self, data):
        new_data = MyData()
        for key, item in data:
            new_data[key] = item
        return new_data
class GraphDataset(InMemoryDataset):
    
    def __init__(self, name, root, data_path, bias_threshold, transform=None, pre_transform=None,
                 pre_filter=None):
        self.path_data = data_path
        super(GraphDataset, self).__init__(root, transform, pre_transform, pre_filter)
        self.pre_transform = None
        self.pre_filter = None
        self.name = name
        self.data, self.slices = torch.load("torch_data/SC")
        
        self.bias_threshold = bias_threshold
        global global_name
        global global_data_path
    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return []

    def download(self):
        pass
    
    def process(self):
        print("Preprocessing.")

        data_list = []
        num_graphs = len(os.listdir(self.path_data))

        # Iterate over instance files and create data objects.
        for num, dirname in enumerate(os.listdir(self.path_data)):
            print(num)
            filename = self.path_data + dirname+ "/problem.lp"
            problem = ecole.scip.Model.from_file(filename).as_pyscipopt()
            data = Data()

            #  Maps networkx ids to new variable node ids.
            node_to_varnode = {}
            #  Maps networkx ids to new constraint node ids.
            node_to_connode = {}

            # Number of variables.
            num_nodes_var = 0
            # Number of constraints.
            num_nodes_con = 0
            # Targets (classes).
            y = []
            y_real = []
            # Features for variable nodes.
            feat_var = []
            # Feature for constraints nodes.
            feat_con = []
            # Right-hand sides of equations.
            feat_rhs = []

            index = []
            index_var = []
            obj = []

            label_file = open(self.path_data + dirname+"/label.json","rb")
            aux = json.load(label_file)
            bestSol = aux["Best_Solution"]
            data_label = np.array(bestSol)
            label_file.close()

            edges_file = open(self.path_data + dirname+"/edges_features.json","rb")
            aux = json.load(edges_file)
            indiceVars,indiceCons,values = np.array(aux["indices"][1]),np.array(aux["indices"][0]),np.array(aux["values"])
            edges_file.close()

            # Iterate over nodes, and collect features.
            for i, var in enumerate(problem.getVars()):
                # Node is a variable node.
                node_to_varnode[i] = num_nodes_var
                num_nodes_var += 1

                y_real.append(bestSol[i])
                if (bestSol[i] < bias_threshold):
                    y.append(0)
                else:
                    y.append(1)

                feat_var.append([var.getObj(),len(np.where(indiceVars == i)[0])])
                obj.append([var.getObj()])

            # Node is constraint node.
            for i, cons in enumerate(problem.getConss()):
                node_to_connode[i] = num_nodes_con
                num_nodes_con += 1

                rhs = problem.getLhs(cons)
                feat_rhs.append([rhs])
                feat_con.append([rhs, len(np.where(indiceCons == i)[0])])
                # feat_con.append([rhs])
                index.append(0)

             # Edge list for var->con graphs.
            edge_list_var = []
            # Edge list for con->var graphs.
            edge_list_con = []

            # Create features matrices for variable nodes.
            edge_features_var = []
            # Create features matrices for constraint nodes.
            edge_features_con = []

            # Remark: graph is directed, i.e., each edge exists for each direction.
            # Flow of messages: source -> target.
            for i,(var,cons) in enumerate(zip(indiceVars,indiceCons)):
                # Source node is con, target node is var.

                # Source node is constraint. C->V.
                edge_list_con.append([node_to_connode[cons], node_to_varnode[var]])
                edge_features_con.append(var)

                # Source node is variable. V->C.
                edge_list_var.append([node_to_varnode[var], node_to_connode[cons]])
                edge_features_var.append(var)
            data.edge_features_con = torch.from_numpy(np.array(edge_features_con)).to(torch.float)
            data.edge_features_var = torch.from_numpy(np.array(edge_features_var)).to(torch.float)
            edge_index_var = torch.tensor(edge_list_var).t().contiguous()
            edge_index_con = torch.tensor(edge_list_con).t().contiguous()

            # Create data object.
            data.edge_index_var = edge_index_var
            data.edge_index_con = edge_index_con

            data.y = torch.from_numpy(np.array(y)).to(torch.long)
            data.y_real = torch.from_numpy(np.array(y_real)).to(torch.float)
            data.var_node_features = torch.from_numpy(np.array(feat_var)).to(torch.float)
            data.con_node_features = torch.from_numpy(np.array(feat_con)).to(torch.float)
            data.rhs = torch.from_numpy(np.array(feat_rhs)).to(torch.float)
            data.obj = torch.from_numpy(np.array(obj)).to(torch.float)
            data.num_nodes_var = num_nodes_var
            data.num_nodes_con = num_nodes_con
            data.index = torch.from_numpy(np.array(index)).to(torch.long)
            data.index_var = torch.from_numpy(np.array(index_var)).to(torch.long)

            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), "torch_data/SC")
# pathr = os.getcwd()
# train_dataset = GraphDataset("SC", pathr+"/processed/", "DataSetMIPGNN/", bias_threshold,transform=MyTransform()).shuffle()

In [91]:

test_scores = []
data_train = "DataSet_SC/"
data_test = "DataSetMIPGNN/"
# data_valide = "DataSet_SC/"


# for rep in [0, 1, 2, 3, 4]:
#     for i in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]:
#         # Bias.
#         for bias in [0.0, 0.001, 0.1]:
rep = 1
bias = 0.1
# GNN.
log = []

# Setup model.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleNet(hidden=64, num_layers=4, aggr="mean", regression=False).to(device)
model_name = "ECS_" + "SC" + str(bias) + "_" + str(rep)
print(model_name, bias, "SC")

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
                                                       factor=0.8, patience=10,
                                                       min_lr=0.0000001)

# Prepare data.
bias_threshold = bias

batch_size = 10

num_epochs = 30

pathr = os.getcwd()+"/processed/"

pd = path_train = path_trainpath_train = data_train
name = name_train = "SC"
graph_dataset = GraphDataset(name_train, pathr, path_train, bias_threshold,
                             transform=MyTransform()).shuffle()


pd = path_test = path_testpath_test = data_test
name = name_test = "SC"
test_dataset = GraphDataset(name_test, pathr, path_test, bias_threshold,
                            transform=MyTransform()).shuffle()

train_index, val_index = train_test_split(list(range(0, len(graph_dataset))), test_size=0.2,train_size=0.8)
val_dataset = graph_dataset[val_index].shuffle()
train_dataset = graph_dataset[train_index].shuffle()
test_dataset = test_dataset.shuffle()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [99]:
print(next(iter(train_loader)))

MyDataBatch(edge_features_con=[3600000], edge_features_var=[3600000], edge_index_var=[2, 3600000], edge_index_con=[2, 3600000], y=[12000], y_real=[12000], var_node_features=[12000, 2], con_node_features=[10000, 2], rhs=[10000, 1], obj=[12000, 1], num_nodes_var=[10], num_nodes_con=[10], index=[10000], index_var=[0], batch=[12000], ptr=[11])


/tmp/ipykernel_3776/3411642700.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.num_nodes_con)
/tmp/ipykernel_3776/3411642700.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.num_nodes_var)


In [100]:
def train(epoch):
    model.train()

    # loss_all = 0
    zero = torch.tensor([0]).to(device)
    one = torch.tensor([1]).to(device)

    loss_all = 0

    for data in train_loader:
        data = data.to(device)

        y = data.y_real
        y = torch.where(y <= bias_threshold, zero, one).to(device)

        optimizer.zero_grad()
        output = model(data)

        loss = F.nll_loss(output, y)
        loss.backward()
        loss_all += batch_size * loss.item()
        optimizer.step()

    return loss_all / len(train_dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    zero = torch.tensor([0]).to(device)
    one = torch.tensor([1]).to(device)
    f1 = F1(num_classes=2, average="macro").to(device)
    pr = Precision(num_classes=2, average="macro").to(device)
    re = Recall(num_classes=2, average="macro").to(device)
    acc = Accuracy(num_classes=2).to(device)

    first = True
    for data in loader:
        data = data.to(device)
        pred = model(data)

        y = data.y_real

        y = torch.where(y <= bias_threshold, zero, one).to(device)
        pred = pred.max(dim=1)[1]

        if not first:
            pred_all = torch.cat([pred_all, pred])
            y_all = torch.cat([y_all, y])
        else:
            pred_all = pred
            y_all = y
            first = False

    return acc(pred_all, y_all), f1(pred_all, y_all), pr(pred_all, y_all), re(pred_all, y_all)


best_val = 0.0
test_acc = 0.0
test_f1 = 0.0
test_re = 0.0
test_pr = 0.0
for epoch in range(1, num_epochs + 1):

    train_loss = train(epoch)
    train_acc, train_f1, train_pr, train_re = test(train_loader)

    val_acc, val_f1, val_pr, val_re = test(val_loader)
    scheduler.step(val_acc)
    lr = scheduler.optimizer.param_groups[0]['lr']

    if val_acc > best_val:
        best_val = val_acc
        test_acc, test_f1, test_pr, test_re = test(test_loader)
        torch.save(model.state_dict(), "./model_new_reps/" + model_name)

    log.append(
        [epoch, train_loss, train_acc, train_f1, train_pr, train_re, val_acc, val_f1, val_pr, val_re,
         best_val, test_acc, test_f1, test_pr, test_re])

    # Break if learning rate is smaller 10**-6.
    if lr < 0.000001 or epoch == num_epochs:
        print([model_name, test_acc, test_f1, test_pr, test_re])
        test_scores.append([model_name, test_acc, test_f1, test_pr, test_re])
        log = np.array(log)
        np.savetxt("./model_new_reps/" + model_name + ".log", log, delimiter=",",
                   fmt='%1.5f')
        break

torch.cuda.empty_cache()


/tmp/ipykernel_3776/3411642700.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.num_nodes_con)
/tmp/ipykernel_3776/3411642700.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(self.num_nodes_var)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3600000 and 1x64)